<a href="https://colab.research.google.com/github/ArimaRX/codeptit/blob/main/colab/cart_pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up environment and import

In [1]:
%%capture
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg
!pip3 install pyvirtualdisplay
!pip3 install -U colabgymrender

In [2]:
import gym
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display
from collections import deque
import tensorflow as tf
import random
import numpy as np

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3055616/45929032 bytes (6.7%)6258688/45929032 bytes (13.6%)9838592/45929032 bytes (21.4%)13991936/45929032 bytes (30.5%)18038784/45929032 bytes (39.3%)22208512/45929032 bytes (48.4%)26066944/45929032 bytes (56.8%)30277632/45929032 bytes (65.9%)34373632/45929032 bytes (74.8%)38559744/45929032 bytes (84.0%)42680320/45929032 bytes (92.9%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root/.

In [3]:
display = Display(visible=0, size=(400, 300))
display.start();
env = gym.make('CartPole-v1')
env = Recorder(env, './video', fps=60)

#global variable

In [4]:
REPLAY_MEMORY_CAPACITY = 50000
NUM_EPISODES = 50
EPSILON = 1
EPSILON_DECAY = 0.9975
REPLAY_MEMORY_BATCH_SIZE = 35
GAMMA = 0.99
TARGET_MODEL_COUNT_MAX = 5
MIN_EPSILON = 0.1

#agent

In [5]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(env.observation_space.shape)))
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dense(2, activation='linear'))
  model.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
  return model

In [6]:
class Agent:
  def __init__(self):
    self.model = create_model()
    self.target_model = create_model()
    self.replay_memory = deque(maxlen = REPLAY_MEMORY_CAPACITY)
    self.target_model_count = 0
  def update_replay_memory(self, state, action, reward, new_state, done):
    if state is None:
      return
    self.replay_memory.append(np.array([state, action, reward, new_state, done]))
  def train(self):
    if len(self.replay_memory) < REPLAY_MEMORY_BATCH_SIZE:
      return
    memory_batch = random.sample(self.replay_memory, k=REPLAY_MEMORY_BATCH_SIZE)
    state_batch = np.array([i[0] for i in memory_batch])
    new_state_batch = np.array([i[3] for i in memory_batch])
    q_val_batch = self.model.predict(tf.constant(state_batch))
    new_q_val_batch = self.target_model.predict(tf.constant(new_state_batch))
    for index, (state, action, reward, new_state, done) in zip(range(len(memory_batch)),memory_batch):
      if not done:
        q_val_batch[index][action] = reward + GAMMA * np.max(new_q_val_batch[index])
      else:
        q_val_batch[index][action] = reward
    self.model.fit(state_batch, q_val_batch)
    self.target_model_count += 1
    if self.target_model_count >= TARGET_MODEL_COUNT_MAX:
      self.target_model_count = 0
      self.target_model.set_weights(self.model.get_weights())

In [9]:
CPModel = Agent()
CPModel.model = tf.keras.models.load_model('/content/drive/MyDrive/model/cart_pole/MyModel')
CPModel.target_model = CPModel.model

#train

In [10]:
for episode in range(NUM_EPISODES):
  env.reset()
  state = None
  done = False
  print('#episode: ', episode)
  while not done:
    if np.random.random() <= EPSILON or state is None:
      action = env.action_space.sample()
    else:
      action = np.argmax(CPModel.model.predict(np.expand_dims(state, axis=0)))
    new_state, reward, done, _ = env.step(action)
    CPModel.update_replay_memory(state, action, reward, new_state, done)
    CPModel.train()
    state = new_state
    if EPSILON > MIN_EPSILON:
        EPSILON *= EPSILON_DECAY
        EPSILON = max(MIN_EPSILON, EPSILON)
  env.play()

#episode:  0


#episode:  1


#episode:  2
2/2 [==============================] - 0s 8ms/step - loss: 17.4675 - accuracy: 0.9714


#episode:  3
2/2 [==============================] - 0s 8ms/step - loss: 21.0028 - accuracy: 0.9429


#episode:  4
2/2 [==============================] - 0s 7ms/step - loss: 22.0709 - accuracy: 0.9143


...

#episode:  45
2/2 [==============================] - 0s 16ms/step - loss: 216.9386 - accuracy: 0.9714


#episode:  46
2/2 [==============================] - 0s 7ms/step - loss: 25.5830 - accuracy: 0.8857


#episode:  47
2/2 [==============================] - 0s 13ms/step - loss: 7.3538 - accuracy: 0.7714


#episode:  48
2/2 [==============================] - 0s 16ms/step - loss: 67.2226 - accuracy: 0.7429


#episode:  49
2/2 [==============================] - 0s 6ms/step - loss: 1.2220 - accuracy: 0.8857


In [11]:
env.reset()
done = False
state = None
while not done:
  if state is None:
    action = env.action_space.sample()
  else:
    action = np.argmax(CPModel.model.predict(np.expand_dims(state, axis=0)))
  state, reward, done, _ = env.step(action)
env.play()

In [12]:
CPModel.model.save('MyModel2')

INFO:tensorflow:Assets written to: MyModel2/assets


In [13]:
!zip -r /content/MyModel2.zip /content/MyModel2

  adding: content/MyModel2/ (stored 0%)
  adding: content/MyModel2/variables/ (stored 0%)
  adding: content/MyModel2/variables/variables.index (deflated 66%)
  adding: content/MyModel2/variables/variables.data-00000-of-00001 (deflated 21%)
  adding: content/MyModel2/assets/ (stored 0%)
  adding: content/MyModel2/keras_metadata.pb (deflated 89%)
  adding: content/MyModel2/saved_model.pb (deflated 88%)


In [14]:
from google.colab import files
files.download('MyModel2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>